In [142]:
import numpy as np
import scipy as sp
from scipy import optimize

Method  Newton Descent



### Intro :

We propose in this part a presentation of newton method for non linear problem solving (second order here) the first part study how we solve a problem without constraint, the second part show **2 methods** for problem solving 
with constraints, those 2 methods are 2 differents ways to look at the same thing

### Part 1 : Newton method (with no constraints)

In one dimensional problem, let consider the function $f(x)$ from $\R \longrightarrow \R$ and $f_T(x)$ the taylor approximation in the point $x_n$ of $f(x_n)$ is : 
$f_T(x_n+\Delta x) \approx f(x_n)+f'(x_n)\Delta x+\frac 1 2 f''(x_n) \Delta x^2$

If $x_{n + 1} = x_n+\Delta x $ is a stationary point, it means that the derivative $\frac{f_T(x_n+\Delta x) - f_T(x_n)}{\Delta x} = \frac{f'(x_n)\Delta x+\frac 1 2 f''(x_n) \Delta x^2}{\Delta x} \implies f'(x_n)\Delta x+\frac 1 2 f''(x_n) \Delta x^2 = 0$
so we could find the $\Delta x$ (the step) either by solving plynomial second order or solving the derivative $f'(x_n)+f'' (x_n) \Delta x$ which give $\Delta x = - \frac{f'(x)}{f''(x)}$

#### Higher dimensions : 

In higher dimension $f'(x) = \nabla f(x)$ the gradient of $f(x)$ and $f''(x) = \mathbf{H}f(x) $ the hessian matrix of $f(x)$ so :

$$\Delta x = \mathbf{H}f(x)^{-1} . \nabla f(x)$$

#### Gradient implementation :

In [143]:
def gradient_f(x, f):
    #f the function
    assert (x.shape[0] >= x.shape[1]), "the vector should be a column vector"
    x = x.astype(np.float64)
    N = x.shape[0]
    eps = abs(np.linalg.norm(x) *  np.finfo(np.float32).eps )
    gradient = []
    f0 = f(x)
    for i in range(N):
        xx0 = 1. * x[i]
        x[i] = x[i] + eps
        f1 = f(x)
        gradient.append(np.asscalar(np.array(f1 - f0))/eps)
        x[i] = xx0
    return np.array(gradient).reshape(x.shape)
    
        
    

#### Test gradient :  

In [144]:
#we consider the function func defined as xT.dot(T) 
def func(x):
    return np.asscalar(x.T.dot(x))

#if x0 =(x1, x2), func(x) = x1² + x2², so grad_f(x0, func) = (2 * x1, 2 * x2)
x0 = np.array([1., 3.]).reshape(2,1)
gradient_f(x0, func)

array([[ 2.00000038],
       [ 6.00000038]])

#### Hessian implementation

In [145]:
def hessian (x, the_func):
    N = x.shape[0]
    x = x.astype(np.float64)
    hessian = np.zeros((N,N)) 
    gd_0 = gradient_f(x, the_func)
    eps = abs(np.linalg.norm(gd_0) * np.finfo(np.float32).eps )
    for i in range(N):
        xx0 = 1.*x[i]
        x[i] = xx0 + eps
        gd_1 =  gradient_f(x, the_func)
        hessian[:,i] = ((gd_1 - gd_0)/eps).reshape(x.shape[0])
        x[i] =xx0
    return hessian

#### Test Hessian :  

In [146]:
#if x0 =(x1, x2), func(x) = x1² + x2²,the hessian matrix should be 2 * I_2 where I_2 is the identity matrix 
#of dimension 2
hessian(x0, func)

array([[  1.99999958e+00,  -1.66692680e-07],
       [ -2.86263754e-07,   1.99999912e+00]])

### Newton algorithm :
Newton algorithm suppose that the sequence $x_{n+1} = x_n + \Delta x$ converge to the stationary point $x*$
so starting from a random point $x_n$ we iterate to $x_{n+1}$ until we reach $x*$

In [147]:
def Newton(func, x0):
    tol = 1e-4
    eps = abs(np.linalg.norm(x0) *  1e-7) 
    x1 = 1. + x0
    while(np.linalg.norm(x1 - x0) > tol):
        x0 = x1
        grad = gradient_f(x0, func)
        x1 = x0 - np.linalg.inv(hessian(x0, func)).dot(grad)
    return x1

#### Test Newton algorithm :

In [148]:
#if x0 =(x1, x2), func(x) = x1² + x2², the newton algorithm applied should give the solution (0, 0)
Newton(func, x0)

array([[ -3.69603615e-09],
       [ -9.02101269e-10]])

#### Comparaison with scipy
We took a scipy example : https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

In [149]:
from scipy.optimize import minimize, rosen, rosen_der, rosen_hess
x1 = [1.3, 0.7, 0.8, 1.9, 1.2]
#Comparaison with scipy
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

def rosen_hess(x):
    x = np.asarray(x)
    H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200*x[0]**2-400*x[1]+2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
    H = H + np.diag(diagonal)
    return H
res = minimize(rosen, x1, method='trust-ncg',
               jac=rosen_der, hess=rosen_hess,
               options={'gtol': 1e-8, 'disp': True})
print("scipy solution is",res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 20
         Hessian evaluations: 19
scipy solution is [ 1.  1.  1.  1.  1.]


In [150]:
res = Newton(rosen, np.array(x1).reshape(5,1))
print("implemented solution is\n",res)

implemented solution is
 [[ 0.99998568]
 [ 0.99997156]
 [ 0.99994342]
 [ 0.99988703]
 [ 0.99977394]]


### Part 2 : Newton method (with constraints)

#### Method 1 :

instead of solving the 1st problem we re-write it by considering the constraint and give the lagrangien function $L(x, \lambda)$
using lagrange multipliers https://en.wikipedia.org/wiki/Lagrange_multiplier, and apply the newton method on 
the lagrangien function and from the solution $(x*, \lambda^*)$ select $x^*$

#### Method 2 : 
Using the method explained here https://www.cs.cmu.edu/~ggordon/10725-F12/scribes/10725_Lecture12.pdf
we solve the system 

\begin{equation}
\begin{pmatrix}
   H_f(x) & J_h(x)^T \\
   J_h(x) & 0 
\end{pmatrix}
\begin{pmatrix}
   \Delta x \\
   \lambda   
\end{pmatrix}
=
\begin{pmatrix}
   -\nabla f(x)\\
    -h(x)
\end{pmatrix}
\end{equation}

where $J_h(x)$ is the jacobian matrix of the constrainsts of f denoted as $h(x)$ 

https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant

so to find our solution :
\begin{equation}
\begin{pmatrix}
   \Delta x \\
   \lambda   
\end{pmatrix}
=
\begin{pmatrix}
   H_f(x) & J_h(x)^T \\
   J_h(x) & 0 
\end{pmatrix}^{-1}
\begin{pmatrix}
   -\nabla f(x)\\
    -h(x)
\end{pmatrix}
\end{equation}

Where $h(x) $ are the function of constraints, they are satisfied where $h(x) = 0$
if we start from a point $x_k$ that satisfy the constraint the system become 

\begin{equation}
\begin{pmatrix}
   \Delta x \\
   \lambda   
\end{pmatrix}
=
\begin{pmatrix}
   H_f(x) & J_h(x)^T \\
   J_h(x) & 0 
\end{pmatrix}^{-1}
\begin{pmatrix}
   -\nabla f(x)\\
    0
\end{pmatrix}
\end{equation}




### 1st Method implem :

We take the introduction example given in wikipedia https://fr.wikipedia.org/wiki/Multiplicateur_de_Lagrange where the aim is to find a portion of the geometric figure which has a volume v0 (constraint) with the minimum surface (function to minimize), our variable are (r:radius,h:height)

Function to minimize : 

$s(r,h) = 2\pi r(r+h)$

Constraint function :

$\phi(r,h) = \pi r^2h - v_0 = 0$

Lagrangien :

$L(r,h,\lambda) = s(r,h) + \lambda\phi(r,h)$


In [194]:
def surface(x):#s(r,h)
    x = np.abs(x)
    return 2 * np.pi * x[0]* x.sum()

In [195]:
def phi_vol(x, v0 = 100):#phi(r,h)
    return np.pi * x[0]**2 * x[1] - v0

In [196]:
def lagrangien(func, N, phis = []): #L(r,h)
    return lambda x: np.asscalar(func(x[:N]) + x[N:].T.dot(np.array([phi(x[:N]) for phi in phis])).reshape(len(x[N:]),1))

In [209]:
#initialize r,h,lambda
c = np.ones(3).reshape(3,1)
#Defining the lagrangien
lag = lagrangien(surface, 2, [phi_vol])

In [210]:
#Newton resolution : 
sol = Newton(lag, c)

### Results and comments : 
-According to Wikipedia the solution should be $h = 2r$ here we have a solution where the first component 
$r = 2.515$ and $h = 5.029$ which seems logic more than that, the evaluation of the constraint = 0 is satisfied

-The algorithm is not too stable when tolerance is too small (1e-6) the algorithm do not converge

In [211]:
print ("2 * r = " ,2 *  sol[0])
print ("h = " , sol[1])
print("the evaluation of the contraint thath should be equal to 0", phi_vol(sol[:2]))

2 * r =  [ 5.03174604]
h =  [ 5.02900248]
the evaluation of the contraint thath should be equal to 0 [ 0.00210852]


### 2d Method implem :

In [200]:
#Defining Jacobian matrix
def jacobian(x, phis):
    jac = []
    for phi in phis:
        jac.append(gradient_f(x, phi).flatten())
    return np.array(jac)

In [201]:
#Defining Matrix Stack
def stack_mat(x, func, phis):
    J = jacobian(x, phis)
    H = hessian(x, func)
    HJ = np.hstack((H,J.T))
    n = J.shape[0]
    Z = np.zeros((n,n))
    JZ = np.hstack((J,Z))
    stack_mat = np.vstack((HJ, JZ))
    return stack_mat

In [202]:
# we consider that the point satisfy the constraints
def delta_x_lambda(x, func, phis):
    n = len(phis)
    return np.linalg.inv(stack_mat(x, func, phis)).dot(-np.vstack((gradient_f(x, func), np.zeros((n,1)))))

In [203]:
#We use the general formula
def delta_x_lambda2(x, func, phis):
    n = len(phis)
    invH = np.linalg.inv(stack_mat(x, func, phis))
    minus_gh = np.vstack((-gradient_f(x, func), - np.array([phi(x) for phi in phis]).reshape(n,1)))
    return  invH.dot(minus_gh)

In [249]:
def Newton2(func, x0, phis):
    tol = 1e-7
    n = x0.shape[0]
    x1 = 1. + x0
    while(np.linalg.norm(x1 - x0) > tol):
        grad = gradient_f(x0, func)
        #The step here is calculated diffrently
       # print("hh")
        dxn = delta_x_lambda2(x0,func, phis)[:n]
       # print(dxn)
        x1 = x0 + dxn
        x0 = x1
    return x1

### Comparaison with scipy

In [250]:
def cons_eq(x):
    return -x[0] + 6 * x[1] - 10
def f(x):
    # x1² + x2
    return x[0] ** 2 + x[1]

In [251]:
#Scipy implementation
A = np.array([-1, 6])
b = 10
A_inv = np.linalg.pinv(A.reshape((A.shape[0], 1)))
#Feaseable point for scipy
x = (b * A_inv).reshape(2,1)
print("constraint val:" ,cons_eq(x))
cons = ({'type': 'eq', 'fun': cons_eq})
sol2 = minimize(f, x, constraints=cons)
print("solution is ",sol2.fun)
print("constraint val:", cons_eq(sol2.x))

constraint val: [  3.55271368e-15]
solution is  1.6597222222222225
constraint val: 1.7763568394e-15


In [252]:
#My implementation with non feaseble point
x0 = np.array([1., 2.5])
print("constraint val:", cons_eq(x0))
sol3 = Newton2(f, x0.reshape(2,1), [cons_eq])
print("solution is ",sol3)
print("constraint val:",cons_eq(sol3))

constraint val: 4.0
solution is  [[-0.0839987 ]
 [ 1.65266688]]
constraint val: [  2.07815809e-09]


### Result and comment
Here we have solved the linear system with newton descent without feaseble point, but this method didn't work for the first example, we d'ont have the result $h = 2 * r$ confirmed(i post the example below), i d'ont know if it an implementation problem or not, the constraint $h(x) = 0$ is verified when we use the general formula by setting the argument sat of the Newton2 function to True

In [257]:
sol2 = Newton2(surface, x0.reshape(2,1), [phi])
print ("2 * r = " ,2 *  sol2[0])
print ("h = " , sol2[1])

2 * r =  [-1.42977294]
h =  [ 1.54751892]
